In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tasks import RoadTypeClfTask, TravelTimeEstimation
from sklearn import metrics, linear_model
from evaluation import Evaluation
from generator import RoadNetwork, Trajectory
import torch
from models import GAEModel, Node2VecModel, Toast
import numpy as np
import networkx as nx

In [ ]:
network = RoadNetwork()
network.load("../osm_data/porto")
data = network.generate_road_segment_pyg_dataset()

trajectory = Trajectory("../datasets/trajectories/Porto/road_segment_map_final.csv")
traj_data = trajectory.generate_TTE_datatset()

In [ ]:
decoder = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])
roadclf = RoadTypeClfTask(decoder, y)
roadclf.register_metric(name="f1_micro", metric_func=metrics.f1_score, args={"average": "micro"})
roadclf.register_metric(name="f1_macro", metric_func=metrics.f1_score, args={"average": "macro"})
roadclf.register_metric(name="f1_weighted", metric_func=metrics.f1_score, args={"average": "weighted"})
roadclf.register_metric(name="accuracy", metric_func=metrics.accuracy_score, args={"normalize": True})
roadclf.register_metric(name="AUC", metric_func=metrics.roc_auc_score, args={"multi_class": "ovo"}, proba=True)

In [ ]:
# 
eidx = network.gdf_edges.iloc[2].name
print(eidx)
fid = network.gdf_edges.iloc[2].fid
node_idx = list(network.line_graph.nodes).index(eidx)
print(fid, eidx, node_idx)


In [ ]:
torch.cuda.set_device(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

travel_time_est = TravelTimeEstimation(traj_dataset=traj_data.iloc[:5000], network=network, device=device, batch_size=128)
travel_time_est.register_metric(name="MSE", metric_func=metrics.mean_squared_error, args={})
travel_time_est.register_metric(name="MAE", metric_func=metrics.mean_absolute_error, args={})
travel_time_est.register_metric(name="RMSE", metric_func=metrics.mean_squared_error, args={"squared": False})

# dw = Node2VecModel(data=data, device=device, p=1, q=1) # p and q from "On Representation Learning for RN"
# dw.load_model(model_path="../model_states/dw/model.pt")
# emb = dw.load_emb()
# travel_time_est.evaluate(emb)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gae = GAEModel(data, device=device) # pseudo init
gae.load_model(model_path="../model_states/gae/model.pt")
node2vec = Node2VecModel(data=data, device=device, p=1, q=4) # p and q from "On Representation Learning for RN"
node2vec.load_model(model_path="../model_states/node2vec/model.pt")
dw = Node2VecModel(data=data, device=device, p=1, q=1) # p and q from "On Representation Learning for RN"
dw.load_model(model_path="../model_states/dw/model.pt")

# toast
dG = network.line_graph
index = np.arange(len(list(dG.nodes))) # index shifted, since 0 and 1 are reserved for mask pad tokens
nx.set_node_attributes(dG, dict(zip(list(dG.nodes), index)), name="index")
toast = Toast(dG, network.gdf_edges)
toast.load_model("../model_states/toast/model_rw_init_epoch12.pt")

In [ ]:
eva = Evaluation()

eva.register_task("Road Type CLF", roadclf)
eva.register_task("Time Travel Estimation", travel_time_est)

eva.register_model("GAE", gae)
eva.register_model("Node2Vec", node2vec)
eva.register_model("DW", dw)
eva.register_model("toast", toast)

res = eva.run()

In [ ]:
print(res[0][0])
res[0][1].head()

In [ ]:
print(res[1][0])
res[1][1].head()